In [2]:
pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.4 MB 495.5 kB/s eta 0:00:03
   - -------------------------------------- 0.1/1.4 MB 563.7 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.4 MB 853.3 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.4 MB 1.1 MB/s eta 0:00:02
   ---------- ----------------------------- 0.4/1.4 MB 1.3 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.4 MB 1.5 MB/s eta 0:00:01
   ---------------- ----------------------- 0.6/1.4 MB 1.6 MB/s eta 0:00:01
   ------------------ --------------------- 0.7/1.4 MB 1.7 MB/s eta 0:00:01
   ------------------------ --------------- 0.9/1.4 MB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.4 MB 2.0 MB/s eta 0:00:01
   ---------------

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [4]:
train_dir = r'C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset'  
test_dir = r'C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset'

In [5]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [6]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [7]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 6400 images belonging to 2 classes.


In [8]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 6400 images belonging to 2 classes.


In [9]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features
train_labels = train_generator.classes

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 31s 155ms/step


In [11]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step


In [13]:
# Replace this with the actual number of classes in your dataset
num_classes = len(np.unique(train_labels))

# Use LightGBM with multiclass classification
lgb_classifier = lgb.LGBMClassifier(
    boosting_type='gbdt',    # Use gradient boosting
    num_leaves=31,           # Maximum leaves in one tree
    max_depth=-1,            # No maximum depth
    learning_rate=0.1,       # Step size
    n_estimators=100,        # Number of trees
    objective='multiclass',  # Multi classification
    num_class=num_classes    # Specify number of classes
)
lgb_classifier.fit(train_features, train_labels)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001859 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11760
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 49
[LightGBM] [Info] Start training from score -1.610219
[LightGBM] [Info] Start training from score -0.222948


LGBMClassifier(num_class=2, objective='multiclass')

In [14]:
# Make predictions on the test data
test_predictions = lgb_classifier.predict(test_features)

In [15]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 100.00%


In [16]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor8.h5')

In [17]:
# Save the trained LightGBM model
joblib.dump(lgb_classifier, 'lgb_classifier.pkl')

['lgb_classifier.pkl']

In [25]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved models
cnn_model = load_model('cnn_feature_extractor8.h5')
dt_classifier = joblib.load('lgb_classifier.pkl')

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for lightgbm

    # Classify the features with the lightgbm model
    prediction = dt_classifier.predict(features)
    
    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented',2: 'NonDemented',3:'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = r'C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset/test/MildDemented/30 (2).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
The image is classified as: MildDemented
